In [92]:
import pandas as pd

In [93]:
df=pd.read_csv('train.tsv',sep='\t',header=0)

In [94]:
print(df.count())

PhraseId      156060
SentenceId    156060
Phrase        156060
Sentiment     156060
dtype: int64


In [95]:
print(df.head(300))

     PhraseId  SentenceId                                             Phrase  \
0           1           1  A series of escapades demonstrating the adage ...   
1           2           1  A series of escapades demonstrating the adage ...   
2           3           1                                           A series   
3           4           1                                                  A   
4           5           1                                             series   
5           6           1  of escapades demonstrating the adage that what...   
6           7           1                                                 of   
7           8           1  escapades demonstrating the adage that what is...   
8           9           1                                          escapades   
9          10           1  demonstrating the adage that what is good for ...   
10         11           1                            demonstrating the adage   
11         12           1               

In [96]:
print(df['Phrase'].head(300))
#pandas访问某一键df['']

0      A series of escapades demonstrating the adage ...
1      A series of escapades demonstrating the adage ...
2                                               A series
3                                                      A
4                                                 series
5      of escapades demonstrating the adage that what...
6                                                     of
7      escapades demonstrating the adage that what is...
8                                              escapades
9      demonstrating the adage that what is good for ...
10                               demonstrating the adage
11                                         demonstrating
12                                             the adage
13                                                   the
14                                                 adage
15                       that what is good for the goose
16                                                  that
17                            w

In [97]:
print(df['Sentiment'].describe())

count    156060.000000
mean          2.063578
std           0.893832
min           0.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           4.000000
Name: Sentiment, dtype: float64


In [98]:
print(df['Sentiment'].value_counts())
#df.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64


In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,recall_score, accuracy_score,classification_report,confusion_matrix
#GridSearchCV被移到了model_selection模块中

In [109]:
X,y=df['Phrase'],df['Sentiment'].as_matrix()
#python的dataframe转换为多维矩阵有两种方法，一种利用as_matrix()属性，一种利用values,
#Dataframe 暴力的写法
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [101]:
pipeline=Pipeline([
    ('vect',TfidfVectorizer(stop_words='english')),
    ('clf',LogisticRegression())
])

In [102]:
parameters={
    'vect__max_df':(0.25,0.5),
    #'vect__ngram_range':((1,1),(1,2)),
    'vect__use_idf':(True,False),
    'clf__C':(0.1,1,10),
     
}

In [103]:
grid_search=GridSearchCV(pipeline,parameters,n_jobs=-1,verbose=1,scoring='accuracy',cv=3)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  2.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.25, 0.5), 'vect__use_idf': (True, False), 'clf__C': (0.1, 1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [104]:
# 固定写法
print('Best score: %0.3f' %grid_search.best_score_)
print('Best parameter set:')
best_parameters=grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s:%r'%(param_name ,best_parameters[param_name]))
    

Best score: 0.627
Best parameter set:
	clf__C:10
	vect__max_df:0.25
	vect__use_idf:False


In [105]:
predictions=grid_search.predict(X_test)

In [107]:
print('Acurracy:%s'% accuracy_score(y_test,predictions))
print('Confusion Matrix:')
print(confusion_matrix(y_test,predictions))
print('Classification Report:')
print(classification_report(y_test,predictions))

Acurracy:0.633910034602
Confusion Matrix:
[[  482   908   320    66     6]
 [  347  2883  3417   361    17]
 [   86  1414 16617  1614    57]
 [   16   259  3395  4059   453]
 [    0    41   247  1259   691]]
Classification Report:
             precision    recall  f1-score   support

          0       0.52      0.27      0.36      1782
          1       0.52      0.41      0.46      7025
          2       0.69      0.84      0.76     19788
          3       0.55      0.50      0.52      8182
          4       0.56      0.31      0.40      2238

avg / total       0.62      0.63      0.62     39015

